In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('/home/com2rng/kaggle/xfeats_dyn')

In [3]:
import torch
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [4]:
from xfeats_dyn.dataset import Coco2017DataModule
from xfeats_dyn.modeling.model import XFeatsDynModule

datamodule = Coco2017DataModule(batch_size=1)


2025-01-27 14:52:16.044 | INFO     | xfeats_dyn.config:<module>:11 - PROJ_ROOT path is: /home/com2rng/kaggle/xfeats_dyn


In [5]:
learning_rate = 3e-4
num_epochs = 10
torch.set_float32_matmul_precision('medium')


In [ ]:
from lightning import Trainer
from torch import nn
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

model = XFeatsDynModule(learning_rate=learning_rate, loss="ppw_mse2", per_pixel_weight=1e5)
early_stopping_cb = EarlyStopping(monitor="train_loss_epoch", min_delta=0.0, patience=10, mode="min", verbose=False, strict=True)
# trainer = Trainer(max_epochs=200, logger=model.tb_logger, overfit_batches=1, log_every_n_steps=1, callbacks=[early_stopping_cb])
trainer = Trainer(max_epochs=200, logger=model.tb_logger, overfit_batches=1, log_every_n_steps=1)
trainer.fit(model, datamodule=datamodule)



Using cache found in /home/com2rng/.cache/torch/hub/verlab_accelerated_features_main
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
2025-01-27 14:55:12.544408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737986112.630512 3153769 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737986112.660007 3153769 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-27 14:55:12.831888: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-c

Setting up stage fit...


In [ ]:
from lightning import Trainer
from torch import nn
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

for loss in [nn.BCELoss(),]:# nn.MSELoss(), nn.SmoothL1Loss(), nn.HuberLoss()]:
    model = XFeatsDynModule(learning_rate=learning_rate, loss=loss)
    early_stopping_cb = EarlyStopping(monitor=f"train_loss_{model.loss_name}", min_delta=0.0, patience=10, mode="min", verbose=False, strict=True)
    trainer = Trainer(max_epochs=num_epochs, logger=model.tb_logger, overfit_batches=1, log_every_n_steps=1, callbacks=[early_stopping_cb])
    trainer.fit(model, datamodule=datamodule)


In [ ]:
trainer.callback_metrics["train_loss_BCELoss"].numpy()

# predictions = trainer.predict(model, datamodule=datamodule, return_predictions=True)

In [ ]:
print(predictions[0][0])

In [10]:
import cv2

def draw_kpts(ref_points, img):

    # Prepare keypoints and matches for drawMatches function
    keypoints1 = [cv2.KeyPoint(p[0], p[1], 5) for p in ref_points]

    out_img = cv2.drawKeypoints(img, keypoints1, 0, color=(0, 255, 0))

    return out_img


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

_, ax = plt.subplots(len(predictions[0]), 1, figsize=(120, 120))

for i, pred in enumerate(predictions[0]):
    kpts = pred["keypoints"]
    image, _ = datamodule.predict_dataset[i]
    image = image.permute(1, 2, 0).numpy()
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    img = draw_kpts(kpts.detach().cpu().numpy(), image)
    ax[i].imshow(img)
    ax[i].axis("off")

plt.show()

In [ ]:
a = [1,2,3,4,5]
print(a[:None])